In [7]:
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Reshape, Dropout
from keras.optimizers import SGD, RMSprop, Adam
from keras import models

In [8]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [9]:
# Fixed, no need change
def load_data(file):
    with open(file, 'rb') as fo:
        try:
            samples = pickle.load(fo)
        except UnicodeDecodeError:  # python 3.x
            fo.seek(0)
            samples = pickle.load(fo, encoding='latin1')

    data, labels = samples['data'], samples['labels']

    data = np.array(data, dtype=np.float32) / 255
    labels = np.array(labels, dtype=np.int32)
    return data, labels

In [10]:
def make_model(num_ch_c1, num_ch_c2, use_dropout):
    model = tf.keras.Sequential()
    model.add(Conv2D(num_ch_c1, kernel_size=9,  padding='valid', activation='relu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding="valid"))
    model.add(Conv2D(num_ch_c2, kernel_size=5, padding='valid', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding="valid"))
    model.add(Flatten())
    model.add(Dense(300))
    
    if use_dropout:
        model.add(Dropout(0.5))
    model.add(Dense(10))
    if use_dropout:
        model.add(Dropout(0.5))
    
    return model

In [5]:
def main(question, optimizer_, num_ch_c1, num_ch_c2, use_dropout):
    seed = 0
    np.random.seed(seed)
    tf.random.set_seed(seed)
    result_dir = "./results"
    model_dir = "./models"
    
    epochs = 1000  # Fixed
    batch_size = 128  # Fixed
    learning_rate = 1e-3

    model = make_model(num_ch_c1, num_ch_c2, use_dropout)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    if optimizer_ == 'SGD':
        optimizer = SGD(learning_rate=learning_rate)
    elif optimizer_ == 'SGD-momentum':  # Question 3(a)
        optimizer = SGD(learning_rate=learning_rate, momentum = 0.1)
    elif optimizer_ == 'RMSprop':  # Question 3(b)
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer_ == 'Adam':  # Question 3(c)
        optimizer = Adam(learning_rate=learning_rate)
    else:
        raise NotImplementedError(f'You do not need to handle [{optimizer_}] in this project.')

    # Training and test
    x_train, y_train = load_data('data_batch_1')
    x_test, y_test = load_data('test_batch_trim')
    
    # Data Preprocessing
    train_x, test_x = [], []
    for img in x_test:
        test_x.append(np.transpose(np.reshape(img,(3, 32,32)), (1,2,0)))
    for img in x_train:
        train_x.append(np.transpose(np.reshape(img,(3, 32,32)), (1,2,0)))
    x_train=np.array(train_x)
    x_test=np.array(test_x)

    # Training
    model.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
    history = model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(x_test, y_test))

    ''' Fill in Question 1(b) here. This website may help:
            https://towardsdatascience.com/visualizing-intermediate-activation-in-convolutional-neural-networks-with-keras-260b36d60d0
    '''

    # Create folder to store models and results
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    if not os.path.exists(result_dir):
        os.mkdir(result_dir)

    # Save model
    if use_dropout:
        model.save(f'{model_dir}/q{question}_{num_ch_c1}_{num_ch_c2}_{optimizer_}_dropout')
    else:
        model.save(f'{model_dir}/q{question}_{num_ch_c1}_{num_ch_c2}_{optimizer_}_no_dropout')

    # Save the plot for losses
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    train_acc = history.history['accuracy']
    test_acc = history.history['val_accuracy']
    plt.plot(range(1, len(train_loss) + 1), train_loss, label='Train')
    plt.plot(range(1, len(val_loss) + 1), val_loss, label='Test')
    plt.title('Model Loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend()
    if use_dropout:
        plt.savefig(
            f'{result_dir}/q{question}_{num_ch_c1}_{num_ch_c2}_{optimizer_}_dropout_loss.png')
    else:
        plt.savefig(
            f'{result_dir}/q{question}_{num_ch_c1}_{num_ch_c2}_{optimizer_}_no_dropout_loss.png'
        )
    plt.close()

    # Save the plot for accuracies
    plt.plot(range(1, len(train_acc) + 1), train_acc, label='Train')
    plt.plot(range(1, len(test_acc) + 1), test_acc, label='Test')
    plt.title('Model Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend()
    if use_dropout:
        plt.savefig(
            f'{result_dir}/q{question}_{num_ch_c1}_{num_ch_c2}_{optimizer_}_dropout_accuracy.png'
        )
    else:
        plt.savefig(
            f'{result_dir}/q{question}_{num_ch_c1}_{num_ch_c2}_{optimizer_}_no_dropout_accuracy.png'
        )
    plt.close()
    
    return model, history.history['val_accuracy'], history.history['val_accuracy'][-1]

In [6]:
#Question 1B
def get_features(model):
    fig_dir = "./figure"
    if not os.path.exists(fig_dir):
        os.mkdir(fig_dir)
    
    #Get first 2 test image
    x_test, _ = load_data('test_batch_trim')
    test_x = []
    for img in x_test:
        test_x.append(np.transpose(np.reshape(img,(3, 32,32)), (1,2,0)))
    test_x=np.array(test_x)
    test = test_x[:2]

    #Create model to take input and output conv1, pool1, conv2, pool2
    layer_output = [layer.output for layer in model.layers[0:4]]

    layer_name = []
    print(f"layer_output consist of:")
    for i in layer_output:
        layer_name.append(str(i).split('"')[1].split('/')[0])
        print(i)

    #Create model for feature map output
    activate_model = models.Model(model.input,layer_output)

    #Predict first 2 test value
    feature_map = activate_model.predict(test)

    #Feature Map shapes
    for layer in feature_map:
        print(layer.shape)
    
    #Save Test Image and Features Map
    imgcnt = 1
    for img in test:
        plt.figure()
        plt.title(f"Test Image {imgcnt}")
        plt.gray()
        plt.axis('off')
        plt.imshow(img)
        plt.savefig(f"{fig_dir}/1b_TestImage{imgcnt}.png")
        plt.close()
        
        featurecnt = 0
        for layer in feature_map:
            plt.figure()
            plt.title(f"Feature Map of {layer_name[featurecnt]}") #NOT SURE WHY NOT PRINTING!
            
            channel = layer.shape[-1]
            for j in range(channel):
                plt.subplot(10, int(channel/10), j+1)
                plt.axis('off')
                plt.imshow(layer[imgcnt-1,:,:,j], cmap='viridis')
            plt.savefig(f"{fig_dir}/1b_TestImage{imgcnt}_{layer_name[featurecnt]}.png")
            plt.close()
            featurecnt+=1
        imgcnt+=1
    print(f"Features Map Generated. Image saved at ./figures/")

In [7]:
if __name__ == '__main__':
    grid_dir = "./grid_results"
    if not os.path.exists(grid_dir):
        os.mkdir(grid_dir)
    
    #Fill in the list with question you want to do. Question 3 have to complete Question 2
    #Other values is for testing models
    question = [1,2,3]
    acc = {}
    
    for i in question:
        if i == 1:
            c1 = 50
            c2 = 60
            opt = "SGD"
            model, data, val_acc = main(i, opt, c1, c2, use_dropout=False)
            acc["q"+str(i)+"_"+str(c1)+"_"+str(c2)] = val_acc
            
            #Question 1B
            get_features(model)
                
        elif i == 2:
            #Grid Search for c1 and c2
            Q2_c1 = [10,30,50,70,90]
            Q2_c2 = [20,40,60,80,100]
            opt = "SGD"

            #Question 2
            for c1 in Q2_c1:
                datalist=[]
                for c2 in Q2_c2:
                    _, data, val_acc = main(i, opt, c1, c2, use_dropout=False)
                    acc["q"+str(i)+"_"+str(c1)+"_"+str(c2)] = val_acc
                    datalist.append(data)
                    
                #Plot validation accuracy and save
                plt.title(f'Model Validation Accuracy for {opt}_C1={c1}')
                plt.ylabel('Val_Accuracy')
                plt.xlabel('epochs')
                for j in range(len(datalist)):
                    plt.plot(datalist[j], label=f'{opt}_{c1}_{Q2_c2[j]}')
                plt.legend(loc="best")
                plt.savefig(f"{grid_dir}/q{i}_{opt}_{c1}.png")
                plt.close()
            
        elif i == 3:
            #Obtain optimal combination channel for c1 and c2
            value =  {k:v for k, v in acc.items() if k.split("_")[0] == "q2"}
            c1 = int(max(value, key=value.get).split("_")[1])
            c2 = int(max(value, key=value.get).split("_")[2])

            #Declare type of optimizer
            optimizers = ["SGD", "SGD-momentum", "RMSprop", "Adam"]
            datalist=[]
            
            #Question 3A, 3B, 3C, 3D
            for optimizer in optimizers:
                _, data, val_acc = main(i, optimizer, c1, c2, use_dropout=False)
                acc["q"+str(i)+"_"+str(c1)+"_"+str(c2)] = val_acc
                datalist.append(data)
                
            _, data, val_acc = main(i, "SGD", c1, c2, use_dropout=True)
            acc["q"+str(i)+"_"+str(c1)+"_"+str(c2)] = val_acc
            datalist.append(data)
            optimizers.append("SGD_w_dropout")
            
            #Plot validation accuracy and save
            plt.title(f'Model Validation Accuracy for Models')
            plt.ylabel('Val_Accuracy')
            plt.xlabel('epochs')
            for k in range(len(datalist)):
                plt.plot(datalist[k], label=f'{optimizers[k]}_{c1}_{c2}')
            plt.legend(loc="best")
            plt.savefig(f"q{i}_Models_{c1}_{c2}.png")
            plt.close()
            
        else:
            #Test model
            c1=70
            c2=80
            model, data, val_acc = main(i, "Adam", c1, c2, use_dropout=True)
            acc["q"+str(i)+"_"+str(c1)+"_"+str(c2)] = val_acc
    
    #Export val_accuracy to excel for comparison
    acc_models3 = pd.Series(acc, name='Val_Accuracy')
    acc_models3.to_excel("Models_Accuracy.xlsx")

Epoch 1/1000
 5/79 [>.............................] - ETA: 6s - loss: 2.3036 - accuracy: 0.0875

KeyboardInterrupt: 